In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('../Dataset/Kaggle/titanic.csv')
data.info()

2025-10-09 18:25:01.970181: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [2]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.20.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
Y = data.iloc[:,1]
X = data[['Pclass','Fare','Sex','Age']]
print(Y)
X

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


,Pclass,Fare,Sex,Age
0,3,7.2500,male,22.0
1,1,71.2833,female,38.0
2,3,7.9250,female,26.0
3,1,53.1000,female,35.0
4,3,8.0500,male,35.0
...,...,...,...,...
886,2,13.0000,male,27.0
887,1,30.0000,female,19.0
888,3,23.4500,female,NaN
889,1,30.0000,male,26.0


In [4]:
Age_median = X['Age'].median()
X['Age'] = X['Age'].fillna(Age_median)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['Sex'] = le.fit_transform(X['Sex'])

fare_95 = X['Fare'].quantile(0.95)
X['Fare'] = np.where(X['Fare'] > fare_95, fare_95, X['Fare'])

X['Fare'] = np.log1p(1+X['Fare'])


/tmp/ipykernel_25046/3313877963.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Age'] = X['Age'].fillna(Age_median)
/tmp/ipykernel_25046/3313877963.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Sex'] = le.fit_transform(X['Sex'])
/tmp/ipykernel_25046/3313877963.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [5]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
x_train

,Pclass,Fare,Sex,Age
331,1,3.417727,1,45.5
733,2,2.708050,1,23.0
382,3,2.295057,1,32.0
704,3,2.287898,1,26.0
813,3,3.504806,0,6.0
...,...,...,...,...
106,3,2.266958,0,21.0
270,1,3.496508,1,28.0
860,3,2.779335,1,41.0
435,1,4.736893,0,14.0


In [ ]:
#creating the layers for Neural Network
model = keras.Sequential(
    [
        keras.layers.Input(shape=(x_train.shape[1],)),
        layers.Dense(1,activation='sigmoid')                            
    ]
)

#summary how neural network looks like
model.summary()

#compiling the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

#training the model
model.fit(x_train,y_train,epochs=500)

I0000 00:00:1760034418.207349   25046 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2240 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5 (20.00 B)

 Trainable params: 5 (20.00 B)

...

Epoch 1/500


2025-10-09 18:27:15.328086: I external/local_xla/xla/service/service.cc:163] XLA service 0x7141a00069e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-09 18:27:15.329877: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 Ti, Compute Capability 7.5
2025-10-09 18:27:15.467462: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-09 18:27:16.080579: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91301


21/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3648 - loss: 11.4908

I0000 00:00:1760034436.960655   25872 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.3750 - loss: 11.2587
Epoch 2/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3750 - loss: 10.7397
Epoch 3/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3708 - loss: 10.2246
Epoch 4/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.3694 - loss: 9.6987
Epoch 5/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3694 - loss: 9.1782
Epoch 6/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3694 - loss: 8.6586
Epoch 7/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3722 - loss: 8.1368
Epoch 8/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3722 - loss: 7.6192
Epoch 9/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3722 - loss: 7.1050
Epoch 10/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3736 - loss: 6.5925
Epoch 11/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3736 - loss: 6.0786
Epoch 12/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3680 

In [ ]:
model.get_weights()

[array([[-0.48522   ],
        [ 0.5173491 ],
        [-2.3375955 ],
        [-0.01081978]], dtype=float32),
 array([0.76931715], dtype=float32)]

In [ ]:
y_pred=model.predict(x_test)
for i in range(20,40):
    if y_pred[i]>0.5:
        print(1)
    elif i%10==0:
        print("-------------------------------\n")
    else:   
        print(0)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step
-------------------------------

0
1
0
0
0
0
0
0
1
-------------------------------

1
0
1
0
0
0
1
1
0


In [ ]:
y_test.iloc[20:40]

294    0
447    1
192    1
682    0
538    0
819    0
30     0
673    1
63     0
396    0
877    0
141    1
793    0
235    0
204    1
620    0
23     1
198    1
842    1
739    0
Name: Survived, dtype: int64

In [ ]:
model.evaluate(x_test,y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - accuracy: 0.7821 - loss: 0.4411


[0.44110822677612305, 0.7821229100227356]